In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
import dgl.nn as dglnn
import dgl
from dgl import from_networkx
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

D:\DoAn\.venv\Lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [3]:
import torch

print(torch.__version__)
print(torch.version.cuda)
if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using GPU.")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

2.3.0+cu121
12.1
CUDA is available. PyTorch is using GPU.
Device count: 1
Current device: 0
Device name: NVIDIA GeForce GTX 1650


In [4]:
data = pd.read_csv('data/NF-UNSW-NB15.csv')
print(data.duplicated().sum(), "fully duplicate rows to remove")
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True, drop=True)

19740 fully duplicate rows to remove


In [5]:
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,149.171.126.0,62073,59.166.0.5,56082,6,0.0,9672,416,11,8,25,15,0,Benign
1,149.171.126.2,32284,59.166.0.5,1526,6,0.0,1776,104,6,2,25,0,0,Benign
2,149.171.126.0,21,59.166.0.1,21971,6,1.0,1842,1236,26,22,25,1111,0,Benign
3,59.166.0.1,23800,149.171.126.0,46893,6,0.0,528,8824,10,12,27,124,0,Benign
4,59.166.0.5,63062,149.171.126.2,21,6,1.0,1786,2340,32,34,25,1459,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603373,59.166.0.2,1640,149.171.126.8,53,17,0.0,130,162,2,2,0,0,0,Benign
1603374,59.166.0.2,3610,149.171.126.6,21,6,1.0,2044,2404,36,34,26,0,0,Benign
1603375,59.166.0.2,4667,149.171.126.6,40725,6,0.0,320,1918,6,8,27,0,0,Benign
1603376,59.166.0.2,5641,149.171.126.6,56243,6,0.0,528,8824,10,12,27,0,0,Benign


In [6]:
data.columns

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label', 'Attack'],
      dtype='object')

In [7]:
data.Attack.value_counts()

Attack
Benign            1531166
Exploits            24727
Fuzzers             19330
Reconnaissance      12275
Generic              5555
DoS                  5040
Analysis             1990
Backdoor             1777
Shellcode            1365
Worms                 153
Name: count, dtype: int64

In [8]:
labels = {
    'Dos': ['DoS'],
    'Probe': ['Reconnaissance'],
    'U2R': ['Generic', 'Analysis', 'Exploits', 'Shellcode', 'Fuzzers'],
    'Normal': ['Benign'],
    'R2L': ['Backdoor', 'Worms']
}

In [9]:
mapping = {value: key for key, values in labels.items() for value in values}
data['Attack_Type'] = data['Attack'].map(mapping)

In [10]:
# data.drop(data.loc[data['Attack_Type']=='Normal'].index[:6900000], axis=0, inplace=True)
# data.drop(data.loc[data['Attack_Type']=='Dos'].index[:520000], axis=0, inplace=True)
# data.drop(data.loc[data['Attack_Type']=='Probe'].index[:100000], axis=0, inplace=True)
# data.drop(data.loc[data['Attack_Type']=='R2L'].index[:30000], axis=0, inplace=True)

In [11]:
data.Attack_Type.value_counts()

Attack_Type
Normal    1531166
U2R         52967
Probe       12275
Dos          5040
R2L          1930
Name: count, dtype: int64

In [12]:
data['IPV4_SRC_ADDR'] = data.IPV4_SRC_ADDR.apply(str)
data['L4_SRC_PORT'] = data.L4_SRC_PORT.apply(str)
data['IPV4_DST_ADDR'] = data.IPV4_DST_ADDR.apply(str)
data['L4_DST_PORT'] = data.L4_DST_PORT.apply(str)

In [13]:
data['IPV4_SRC_ADDR'] = data['IPV4_SRC_ADDR'] + ':' + data['L4_SRC_PORT']
data['IPV4_DST_ADDR'] = data['IPV4_DST_ADDR'] + ':' + data['L4_DST_PORT']
data.drop(columns=['L4_SRC_PORT', 'L4_DST_PORT'], inplace=True)

In [14]:
data.drop(columns=['Label', 'Attack'], inplace=True)
data.rename(columns={"Attack_Type": "label"}, inplace=True)


In [15]:
le = LabelEncoder()
le.fit_transform(data.label.values)
data['label'] = le.transform(data['label'])

array([1, 1, 1, ..., 1, 1, 1])

In [16]:
data.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label
0,149.171.126.0:62073,59.166.0.5:56082,6,0.0,9672,416,11,8,25,15,1
1,149.171.126.2:32284,59.166.0.5:1526,6,0.0,1776,104,6,2,25,0,1
2,149.171.126.0:21,59.166.0.1:21971,6,1.0,1842,1236,26,22,25,1111,1
3,59.166.0.1:23800,149.171.126.0:46893,6,0.0,528,8824,10,12,27,124,1
4,59.166.0.5:63062,149.171.126.2:21,6,1.0,1786,2340,32,34,25,1459,1


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    data, data.label, test_size=0.2, random_state=47, stratify=data.label)

In [18]:
target_encoder = ce.TargetEncoder(cols=['TCP_FLAGS', 'L7_PROTO', 'PROTOCOL'])
target_encoder.fit(X_train[['PROTOCOL', 'L7_PROTO', 'TCP_FLAGS']], y_train)

X_train[['PROTOCOL', 'L7_PROTO', 'TCP_FLAGS']] = target_encoder.transform(
    X_train[['PROTOCOL', 'L7_PROTO', 'TCP_FLAGS']], y_train)
X_test[['PROTOCOL', 'L7_PROTO', 'TCP_FLAGS']] = target_encoder.transform(X_test[['PROTOCOL', 'L7_PROTO', 'TCP_FLAGS']],
                                                                         y_test)


D:\DoAn\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
D:\DoAn\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
D:\DoAn\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Cal

TargetEncoder(cols=['TCP_FLAGS', 'L7_PROTO', 'PROTOCOL'])

D:\DoAn\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
D:\DoAn\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
D:\DoAn\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Cal

In [19]:
scaler = StandardScaler()
scaler.fit(X_train.iloc[:, 2:-1])

X_train.iloc[:, 2:-1] = scaler.transform(X_train.iloc[:, 2:-1])
X_train['h'] = X_train.iloc[:, 2:-1].values.tolist()

X_test.iloc[:, 2:-1] = scaler.transform(X_test.iloc[:, 2:-1])
X_test['h'] = X_test.iloc[:, 2:-1].values.tolist()


StandardScaler()

C:\Users\houta\AppData\Local\Temp\ipykernel_19620\1556601882.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.03152672  0.04613996 -0.06742953 ... -0.02780565 -0.02110684
 -0.06259294]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 2:-1] = scaler.transform(X_train.iloc[:, 2:-1])
C:\Users\houta\AppData\Local\Temp\ipykernel_19620\1556601882.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.15971071 -0.24259723 -0.24340483 ... -0.08772102 -0.21328495
 -0.24268823]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:, 2:-1] = scaler.transform(X_train.iloc[:, 2:-1])
C:\Users\houta\AppData\Local\Temp\ipykernel_19620\1556601882.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future err

In [20]:
data

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label
0,149.171.126.0:62073,59.166.0.5:56082,6,0.0,9672,416,11,8,25,15,1
1,149.171.126.2:32284,59.166.0.5:1526,6,0.0,1776,104,6,2,25,0,1
2,149.171.126.0:21,59.166.0.1:21971,6,1.0,1842,1236,26,22,25,1111,1
3,59.166.0.1:23800,149.171.126.0:46893,6,0.0,528,8824,10,12,27,124,1
4,59.166.0.5:63062,149.171.126.2:21,6,1.0,1786,2340,32,34,25,1459,1
...,...,...,...,...,...,...,...,...,...,...,...
1603373,59.166.0.2:1640,149.171.126.8:53,17,0.0,130,162,2,2,0,0,1
1603374,59.166.0.2:3610,149.171.126.6:21,6,1.0,2044,2404,36,34,26,0,1
1603375,59.166.0.2:4667,149.171.126.6:40725,6,0.0,320,1918,6,8,27,0,1
1603376,59.166.0.2:5641,149.171.126.6:56243,6,0.0,528,8824,10,12,27,0,1


In [21]:
G = nx.from_pandas_edgelist(
    X_train,
    source='IPV4_SRC_ADDR',
    target='IPV4_DST_ADDR',
    edge_attr=['h', 'label'],
    create_using=nx.MultiDiGraph()
)

In [22]:
from pyvis.network import Network
nt = Network('500px', '500px')
nt.from_nx(G)
nt.show('nx.html')

KeyboardInterrupt: 

In [ ]:
G = from_networkx(G, edge_attrs=['h', 'label'])

In [45]:
G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1])

In [47]:
G.edata['h'].shape

torch.Size([1282702, 1, 8])

In [46]:
G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype=th.bool)
G.ndata['h'] = th.reshape(G.ndata['h'], (G.ndata['h'].shape[0], 1, G.ndata['h'].shape[1]))
G.edata['h'] = th.reshape(G.edata['h'], (G.edata['h'].shape[0], 1, G.edata['h'].shape[1]))

In [49]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()


class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super(SAGELayer, self).__init__()
        ### force to output fix dimensions
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)
        ### apply weight
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        self.activation = activation

    def message_func(self, edges):
        return {'m': self.W_msg(th.cat([edges.src['h'], edges.data['h']], 2))}

    def forward(self, g_dgl, nfeats, efeats):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
            # Eq4
            g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
            # Eq5          
            g.ndata['h'] = F.relu(self.W_apply(th.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))
            return g.ndata['h']


class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super(SAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(SAGELayer(ndim_in, edim, 128, activation))
        self.layers.append(SAGELayer(128, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, g, nfeats, efeats):
        for i, layer in enumerate(self.layers):
            if i != 0:
                nfeats = self.dropout(nfeats)
            nfeats = layer(g, nfeats, efeats)
        return nfeats.sum(1)


class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        global score
        global emb
        emb = th.cat([h_u, h_v], 1)
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']


class Model(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super().__init__()
        self.gnn = SAGE(ndim_in, ndim_out, edim, activation, dropout)
        self.pred = MLPPredictor(ndim_out, 5)

    def forward(self, g, nfeats, efeats):
        h = self.gnn(g, nfeats, efeats)
        return self.pred(g, h)

In [50]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.unique(G.edata['label'].numpy()),
                                                  y=G.edata['label'].numpy())

In [51]:
class_weights = th.FloatTensor(class_weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [52]:
G = G.to('cuda:0')
G.device

device(type='cuda', index=0)

In [53]:
G.ndata['h'].device
G.edata['h'].device
G.ndata['h'].shape
G.edata['h'].shape

device(type='cuda', index=0)

device(type='cuda', index=0)

torch.Size([925629, 1, 8])

torch.Size([1282702, 1, 8])

In [55]:
node_features = G.ndata['h']
edge_features = G.edata['h']

edge_label = G.edata['label']
train_mask = G.edata['train_mask']

model = Model(G.ndata['h'].shape[2], 128, G.ndata['h'].shape[2], F.relu, 0.2).cuda()
opt = th.optim.Adam(model.parameters())

for epoch in range(1, 2000):
    pred = model(G, node_features, edge_features).cuda()
    loss = criterion(pred[train_mask], edge_label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch % 100 == 0:
        print('Epoch:', epoch, 'Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.22 GiB. GPU 

In [ ]:
model

In [24]:
G_test = nx.from_pandas_edgelist(
    X_test,
    source='IPV4_SRC_ADDR',
    target='IPV4_DST_ADDR',
    edge_attr=['h', 'label'],
    create_using=nx.MultiDiGraph()
)


In [ ]:
G_test = from_networkx(G_test, edge_attrs=['h', 'label'])
actual = G_test.edata.pop('label')

G_test.ndata['feature'] = th.ones(G_test.num_nodes(), G.ndata['h'].shape[2])
G_test.ndata['feature'] = th.reshape(G_test.ndata['feature'],
                                     (G_test.ndata['feature'].shape[0], 1, G_test.ndata['feature'].shape[1]))
G_test.edata['h'] = th.reshape(G_test.edata['h'], (G_test.edata['h'].shape[0], 1, G_test.edata['h'].shape[1]))

In [25]:
from pyvis.network import Network
nt = Network('500px', '500px')
nt.from_nx(G_test)
nt.show('nx.html')

KeyboardInterrupt: 

In [ ]:
G_test = G_test.to('cuda:0')

In [ ]:
import timeit

start_time = timeit.default_timer()
node_features_test = G_test.ndata['feature']
edge_features_test = G_test.edata['h']
test_pred = model(G_test, node_features_test, edge_features_test).cuda()
elapsed = timeit.default_timer() - start_time

In [ ]:
test_pred = test_pred.argmax(1)
test_pred = th.Tensor.cpu(test_pred).detach().numpy()
actual = le.inverse_transform(actual)
test_pred = le.inverse_transform(test_pred)

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

plot_confusion_matrix(cm=confusion_matrix(actual, test_pred),
                      normalize=False,
                      target_names=np.unique(actual),
                      title="Confusion Matrix")
from sklearn.metrics import classification_report

target_names = np.unique(actual)
print(classification_report(actual, test_pred, target_names=target_names, digits=4))
actual
test_pred

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

accuracy = accuracy_score(actual, test_pred)
f1 = f1_score(actual, test_pred, average='weighted')  # average='weighted' để tính theo tỷ trọng
recall = recall_score(actual, test_pred, average='weighted')

# In kết quả
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')


In [ ]:
import dgl

dgl_graph = dgl.from_networkx(G)

In [ ]:
protocol = torch.tensor(data['PROTOCOL'].values)
l7_proto = torch.tensor(data['L7_PROTO'].values)
in_bytes = torch.tensor(data['IN_BYTES'].values)
out_bytes = torch.tensor(data['OUT_BYTES'].values)
in_pkts = torch.tensor(data['IN_PKTS'].values)
out_pkts = torch.tensor(data['OUT_PKTS'].values)
tcp_flags = torch.tensor(data['TCP_FLAGS'].values)
flow_duration = torch.tensor(data['FLOW_DURATION_MILLISECONDS'].values)
label = torch.tensor(data['label'].values)

In [ ]:
dgl_graph.edata.update({
    'protocol': protocol,
    'l7_proto': l7_proto,
    'in_bytes': in_bytes,
    'out_bytes': out_bytes,
    'in_pkts': in_pkts,
    'out_pkts': out_pkts,
    'tcp_flags': tcp_flags,
    'flow_duration': flow_duration,
    'label': label
})
print("DGLGraph Edges:", dgl_graph)


In [ ]:
import dgl.nn.pytorch as dgl_nn

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GraphSAGE, self).__init__()
        # Thêm tham số aggregator_type
        self.conv1 = dglnn.SAGEConv(in_feats, hidden_feats, aggregator_type='mean', activation=F.relu)
        self.conv2 = dglnn.SAGEConv(hidden_feats, out_feats, aggregator_type='mean')  # Chọn lại kiểu tổng hợp ở đây

    def forward(self, g, features):
        h = self.conv1(g, features)
        h = self.conv2(g, h)
        return h

In [ ]:
from torch.optim import Adam

train_mask, test_mask = train_test_split(range(len(data)), test_size=0.2, random_state=42)
train_labels = dgl_graph.edata['label'][train_mask]
test_labels = dgl_graph.edata['label'][test_mask]

# Khởi tạo mô hình, optimizer và loss function
model = GraphSAGE(in_feats=8, hidden_feats=16, out_feats=2)  # in_feats là số lượng đặc trưng đầu vào
optimizer = Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# Huấn luyện mô hình
for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    # Lấy đặc trưng cạnh (ví dụ: sử dụng số bytes hoặc một đặc trưng nào đó)
    features = dgl_graph.edata['in_bytes'].float().view(-1, 1)  # Reshape nếu cần
    output = model(dgl_graph, features)

    # Tính toán loss và cập nhật
    loss = loss_fn(output[train_mask], train_labels)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
labels = torch.tensor(data['label'].values)

In [ ]:
import torch
import torch.nn as nn
import dgl.function as fn
from dgl.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.optim as optim


class GraphSAGEModel(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, num_classes, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = torch.relu(h)
        h = self.conv2(g, h)
        return h


class DGLGraphTrainer:
    def __init__(self, dgl_graph, features, labels, num_classes, hidden_size=16, lr=0.01, epochs=100):
        self.graph = dgl_graph
        self.features = features
        self.labels = labels
        self.num_classes = num_classes
        self.model = GraphSAGEModel(features.shape[1], hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.epochs = epochs

    def train(self, train_idx, train_labels):
        self.model.train()
        for epoch in range(self.epochs):
            # Forward pass
            logits = self.model(self.graph, self.features)
            loss = self.loss_fn(logits[train_idx], train_labels)

            # Backward pass and optimization
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if epoch % 10 == 0:
                print(f'Epoch {epoch}/{self.epochs}, Loss: {loss.item()}')

    def test(self, test_idx, test_labels):
        self.model.eval()
        with torch.no_grad():
            logits = self.model(self.graph, self.features)
            predicted_classes = torch.argmax(logits[test_idx], dim=1)
            accuracy = accuracy_score(test_labels.cpu(), predicted_classes.cpu())
            print(f'Test Accuracy: {accuracy}')
        return accuracy

In [ ]:
model = GCN(
    g=a,
    in_feats=1000,
    n_hidden=200,
    n_classes=2,
    activation=nn.PReLU(),
    dropout=nn.PReLU(),
)

loss_fcn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-3, weight_decay=5e-4
)

for epoch in range(200):
    model.train()
    logits = model(features)
    labels_train = process_classids(labels_temp=labels[train_mask_zs])
    loss_train = loss_fcn(logits[train_mask_zs], labels_train)
    optimizer.zero_grad()
    print(
        "Epoch {:d} | Train Loss {:.5f}".format(
            epoch + 1, loss_train.item()
        )
    )
    loss_train.backward()
    optimizer.step()
model.eval()
embeds = model.embed(features)

In [ ]:
data

In [ ]:

# Kiểm tra các loại cột
columns = data.columns.tolist()
print("Các cột trong dữ liệu:", columns)

# Đếm số lượng loại địa chỉ IP, cổng, v.v.
unique_src_ips = data['IPV4_SRC_ADDR'].nunique()
unique_dst_ips = data['IPV4_DST_ADDR'].nunique()

print(f"Số lượng địa chỉ IP nguồn duy nhất: {unique_src_ips}")
print(f"Số lượng địa chỉ IP đích duy nhất: {unique_dst_ips}")

# Kiểm tra số lượng lớp (label) khác nhau
unique_labels = data['label'].nunique()
print(f"Số lượng lớp khác nhau: {unique_labels}")

# Quyết định sử dụng homograph hay heterograph
if unique_src_ips > 1 or unique_dst_ips > 1:
    print("Dữ liệu này có thể phù hợp với heterograph.")
else:
    print("Dữ liệu này có thể phù hợp với homograph.")

In [ ]:
label_counts = data['label'].value_counts(normalize=True)
print("Tỷ lệ các lớp trước khi lấy mẫu:\n", label_counts)

# Lấy mẫu có tỷ lệ lớp không thay đổi
sampled_data = data.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=1, random_state=42)).reset_index(
    drop=True)

# Giới hạn số lượng mẫu về 1000
if len(sampled_data) > 1000:
    sampled_data = sampled_data.sample(n=1000, random_state=42)
elif len(sampled_data) < 1000:
    print("Số lượng mẫu không đủ để lấy 1000.")

# Kiểm tra tỷ lệ lớp sau khi lấy mẫu
sampled_label_counts = sampled_data['label'].value_counts(normalize=True)
print("Tỷ lệ các lớp sau khi lấy mẫu:\n", sampled_label_counts)

# Lưu mẫu vào file mới nếu cần
sampled_data.to_csv('sampled_data.csv', index=False)

In [ ]:
sampled_data

In [ ]:

# Kiểm tra các loại cột
columns = sampled_data.columns.tolist()
print("Các cột trong dữ liệu:", columns)

# Đếm số lượng loại địa chỉ IP, cổng, v.v.
unique_src_ips = sampled_data['IPV4_SRC_ADDR'].nunique()
unique_dst_ips = sampled_data['IPV4_DST_ADDR'].nunique()

print(f"Số lượng địa chỉ IP nguồn duy nhất: {unique_src_ips}")
print(f"Số lượng địa chỉ IP đích duy nhất: {unique_dst_ips}")

# Kiểm tra số lượng lớp (label) khác nhau
unique_labels = sampled_data['label'].nunique()
print(f"Số lượng lớp khác nhau: {unique_labels}")

# Quyết định sử dụng homograph hay heterograph
if unique_src_ips > 1 or unique_dst_ips > 1:
    print("Dữ liệu này có thể phù hợp với heterograph.")
else:
    print("Dữ liệu này có thể phù hợp với homograph.")